In [36]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
from sqlite3.dbapi2 import Cursor
import sqlite3
import pickle
from numpy.core.fromnumeric import sort
import pandas as pd
import numpy as np
print("Import Done")

Import Done


In [37]:
def connect():
    conn = sqlite3.connect('Users.sqlite')

    return conn

In [38]:
"""
    Fetching Rankings Table and converting into dataframe
"""

conn = connect()
cursor = conn.cursor()
sql_query = "select * from Ratings"
cursor.execute(sql_query)
results = list(cursor)
conn.close()
data = [list(i) for i in results]
df = pd.DataFrame(data = data, columns = [
    "images",
    "users",
    "ratings"
])

In [39]:
"""
    Initializing singular value decomposition (SVD)
"""

reader = Reader()

data = Dataset.load_from_df(df, reader)

svd = SVD()

In [40]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=2, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.2991  1.5669  1.4330  0.1339  
MAE (testset)     0.9786  1.3801  1.1793  0.2007  
Fit time          0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    


{'test_rmse': array([1.29911757, 1.56691976]),
 'test_mae': array([0.97862638, 1.38005328]),
 'fit_time': (0.0020203590393066406, 0.0009996891021728516),
 'test_time': (0.0, 0.0)}

In [41]:
"""
    Fitting Our model with training data
"""
trainset = data.build_full_trainset()
svd.fit(trainset)

In [42]:
"""
    Fetch all images
"""
conn = connect()
cursor = conn.cursor()
sql_query = "SELECT * FROM Designs where City in ('Rawalpindi', 'Islamabad')"
cursor.execute(sql_query)
results = list(cursor)
conn.close()
data = [list(i) for i in results]
df = pd.DataFrame(data = data, columns = [
    'image',
    'city',
    'category',
    'style'
])

In [43]:
df['Estimate_Score'] = df['image'].apply(lambda x: svd.predict(x, 'johny').est)
df = df.sort_values(by=['Estimate_Score'], ascending=False)


In [44]:
df.head().to_dict('records')

[{'image': 'https://kitandkaboodle.com/wp-content/uploads/2021/10/6R3A1133-1-1024x683.jpg',
  'city': 'Rawalpindi',
  'category': 'living_style_5',
  'style': 'living',
  'Estimate_Score': 3.500833334524926},
 {'image': 'https://kitandkaboodle.com/wp-content/uploads/2021/10/6R3A8989-1024x1024.jpg',
  'city': 'Rawalpindi',
  'category': 'living_style_8',
  'style': 'living',
  'Estimate_Score': 3.4217569973654145},
 {'image': 'https://kitandkaboodle.com/wp-content/uploads/2021/10/6R3A7264-683x1024.jpg',
  'city': 'Rawalpindi',
  'category': 'living_style_7',
  'style': 'living',
  'Estimate_Score': 3.185697277010673},
 {'image': 'https://kitandkaboodle.com/wp-content/uploads/2021/10/25-683x1024.jpg',
  'city': 'Rawalpindi',
  'category': 'accessories_style_10',
  'style': 'accessories',
  'Estimate_Score': 3.1581463197986666},
 {'image': 'https://kitandkaboodle.com/wp-content/uploads/2021/10/6R3A0294-683x1024.jpg',
  'city': 'Rawalpindi',
  'category': 'dining_style_3',
  'style': 'dini

In [34]:
def save_model(filename, model):
    # Save to file in the current working directory
    pkl_filename = filename + ".pkl"
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)

In [35]:
save_model('SVD', svd)